# Tiled Primer Design

This jupyter notebook semi-automates the process of designing primers using k-mers for tiled PCR amplification across random sites in a single genome. 


### Credit

Written by Julie Chen and Gowtham Thakku

## Set up 


In [16]:
# The necessary python packages are imported here. If you receive an error, you may have to install these packages

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
import numpy as np
import pandas as pd
import itertools
import re
import xlrd
import random
import seaborn as sns

In [17]:
# declare some modifiable parameters here:

date = '20220713'
kmerSize = 5
genomeFileName = '../NC_000962.fasta'
record = SeqIO.read(genomeFileName, 'fasta')
fastaSeq = record.seq

# note: few more manual inputs later (see Doc for more instructions)

## Designing Primers

### Extract initial primer sequences using a chosen k-mer

**How to tell if a k-mer will likely self-dimerize**: Does the reverse (NOT rc) mostly bind to the forward sequence of the k-mer? If so, don't use this k-mer. \
e.g. F: 5' GCTGC 3' would be R: 3' CGTCG 5' which is complementary at 4/5 positions

In [19]:
# based on the "compatible" k-mers identified, select a set of k-mers to use
df_kmers = pd.read_csv('kmers_diverse_ranked.csv')
df_kmers = df_kmers[df_kmers['num. of occurrences'] >= 1] 


# Specify any sequences here that you do not want the chosen kmers to be complementary to
# (e.g. if you already have primer sequences to include in the pool, list them here so any new kmers don't bind to them)
excludeSeq = ['TCTTTAAAATCGCGTTCGCCCTTCGCAATTCCCTATAGTGAGTCGTATTAATTTC',\
'TCTTTCAGGTCGAGTACGCCTTCTTGTTGGCCCTATAGTGAGTCGTATTAATTTC',\
'TCTTTGGCCATGATCGACACTTGCGACTTGCCCTATAGTGAGTCGTATTAATTTC',\
'AGTACCCGATCATATTGGGCAACAACTGATCCCTATAGTGAGTCGTATTAATTTC',\
'GCTGCCCACTCCGAATCGTGCTGACCGCGGCCCTATAGTGAGTCGTATTAATTTC',\
'GCTGCTCGGCGTCGATAAGATGAGAAGAGGCCCTATAGTGAGTCGTATTAATTTC',\
'GCTGCGTTCGCGGTAGCCCGCCCCGCACAGCCCTATAGTGAGTCGTATTAATTTC',\
'GCTGCCATCAGATTGGCTGCGTAGTGGGTTCCCTATAGTGAGTCGTATTAATTTC',\
'GCTGCTCGGGCTCGGGTGAGGACCTCGAGGCCCTATAGTGAGTCGTATTAATTTC',\
'ACACCCCACCGCAAACCATCGGGGCCCTCGCCCTATAGTGAGTCGTATTAATTTC',\
'ACACCGTAGTTGGCGGCGTGGACGCGGCTGCCCTATAGTGAGTCGTATTAATTTC',\
'CGGCCCCGTCCTCGGCGGAGGTGACCTGGACCCTATAGTGAGTCGTATTAATTTC',\
'CATCGACCTGCGCCTGGCGCACCCACTTACCCCTATAGTGAGTCGTATTAATTTC',\
'CATCGTGGAAGCGACCCGCCAGCCCAGGATCCCTATAGTGAGTCGTATTAATTTC',\
'TGAGCACCAGGGCGTCGGCGGCGAGGAAGGCCCTATAGTGAGTCGTATTAATTTC',\
'GAGCTTCCGACCGCTCCGACCGACGGTTGGCCCTATAGTGAGTCGTATTAATTTC',\
'TTGGTCATCAGCCGTTCGACGGTGCATCTGCCCTATAGTGAGTCGTATTAATTTC',\
'ACCACCTGACATGACCCCATCCTTTCCAAGCCCTATAGTGAGTCGTATTAATTTC',\
'CGGCCAGCACGCTAATTACCCGGTTCATCG',\
'AGTACACATCGATCCGGTTCAGCGAGCGGC',\
'CGGCCCGTATACCTTCCTCGCCGCCGACGC',\
'ACACCGCCCGCACCGACCTGCTGGCGTTCA',\
'GCTGCGCGGAGACGGTGCGTAAGTGGGTGC',\
'GCTGCGCGGGCTGCTCTCGACGTTCATCGC',\
'ACACCCGTGCCGCAACCATCGACGTCGCGA',\
'GCTGCACTCCATCTACGACCAGCCCGACGC',\
'ACACCCCAGCACTGACCACCTAGACTGCCA',\
'CATCGACCTACTACGACCACATCAACCGGG',\
'CATCGAGGTGGCCAGATGCACCGTCGAACG',\
'CGGCCTGTCCGGGGTCGCGCTGGTCACCAG',\
'TGAGCTGAAGCGCTTGCGGCGGGACAACGC',\
'AAGCCATCTGGACCCGCCAACAAGAAGGCG',\
'ACACCTTGATCGCCACCGGCGTCAACGCCG',\
'CGGCCTATACAAGACCGAGCTGATCAAACC',\
'AAGCCCGCAGGACCACGATCGCTGATCCGG',\
'TGAGCGGGCGGTGCGGATGGTCGCAGAGAT']

for currSeq in excludeSeq:
    for i in range(len(currSeq)-5):
        curr5mer = currSeq[i:i+5]
        df_kmers.drop(df_kmers[df_kmers['k-mer sequence']==curr5mer].index, inplace=True)
  

chosenKmers = list(df_kmers['k-mer sequence'])

## If you want to manually specify the list of possible kmers, do so here
# chosenKmers = ['GCATC', 'ATCCG', 'CAGGT', 'TCCAG', 'CGCAT', 'GCTCA', \
#                'CGACT', 'ATCAG', 'TCCGA', 'GACAT', 'TCAGG', 'GCACT', \
#                'GCAAT', 'TCAAG', 'CGCTA', 'GACTC', 'CTCAG', 'GCTAT', \
#                'GACTA', 'GCTAA', 'CTAAG', 'GTTCA', 'GCTAC', 'GCCAT']

#PARAMETERS CAN BE CHANGED
#Set other parameters for run
gRNA_size = 28 # This is the length of the crRNA spacer sequence
wiggleRoom = 20 # This determines the range of possible distances between forward and reverse primers
primerLength = 30 # This is the length of the primers to be designed (can be truncated later if needed)


In [20]:
# function to find all positions of a particular kmer in the genome
def listAllPositions(kmer, genomeSeq = fastaSeq):

    pattern = re.compile(r'(?=(' + kmer + '))')
    positions = [p.start() for p in re.finditer(pattern, str(genomeSeq))]

    return positions

# function to find the reverse complement of a given sequence
def revComplementSeq(seqtoRC):

    seqObject = Seq(seqtoRC)
    rc = seqObject.reverse_complement()

    return rc

# function to find 3'-end positions for compatible forward kmer positions
def find3EndF(kmerIndices):

    ends = [(i + kmerSize) for i in kmerIndices]

    return ends


def gcContentFrac(seqGC):
    gcNum = seqGC.count('G')+seqGC.count('g')+seqGC.count('C')+seqGC.count('c')
    gcFrac = gcNum/len(seqGC)
    
    return gcFrac


#function to find distance between 3' ends of foward and reverse primers
def positionDifference(revList, forList):

    differences = [(revList[r] - forList[r]) for r in range(len(revList))]
    return differences

#function to get forward primer sequences
def initialPrimerSeqF(positionsF, genomeSeq = fastaSeq, length = primerLength-kmerSize):

    primerSequences = []

    for i in positionsF:

        primerStart = i - length
        primerEnd = i + kmerSize
        primerSequences.append(str(genomeSeq[primerStart:primerEnd]))

    return primerSequences


#function to get reverse primer sequences
def initialPrimerSeqR(positionsR, genomeSeq = fastaSeq, length = primerLength-kmerSize):

    primerSequences = []

    for i in positionsR:

        primerEnd = i + kmerSize + length
        rcSeq = revComplementSeq(str(genomeSeq[i:primerEnd]))
        primerSequences.append(str(rcSeq))

    return primerSequences

In [21]:
#function to generate all possible compatible primers for a given set of k-mers

def numCompatiblePrimers(chosenKmers, gRNA_size, wiggleRoom, primerLength, genomeSeq = fastaSeq):
    
    kmerPositionsF = []

    kmerSeqColumn = 'k-mer sequence'

    # loop to identify all compatible positions of kmers within acceptable distance

    for currKmer in chosenKmers:


        positions_currKmer = listAllPositions(currKmer, genomeSeq)
        positions_currKmer = [t for t in positions_currKmer if t >50 and t <2878800]
        compatiblePositions = []

        for currPos in positions_currKmer:

            primerSpace_start = currPos + kmerSize + gRNA_size 
            primerSpace_end = primerSpace_start + wiggleRoom + kmerSize

            if primerSpace_end < len(genomeSeq):
                searchSpace = str(genomeSeq[primerSpace_start:primerSpace_end])

            for searchKmer in chosenKmers:
                kmer_rc = revComplementSeq(searchKmer) 
                if searchSpace.find(str(kmer_rc)) > -1:
                    compatiblePositions.append(currPos)
                    break

        kmerPositionsF.extend(compatiblePositions)

#     print("Number of F primer Positions: ",len(kmerPositionsF))

    positions3F = find3EndF(kmerPositionsF)


    # function to find 3'-end positions for compatible reverse kmer positions
    kmerPositionsR = []

    kmerSeqColumn = 'k-mer sequence'

    revPositions = []
    positions_rc = []
    for searchKmer in chosenKmers:     
        kmer_rc = revComplementSeq(searchKmer)
        positions_rc.extend(listAllPositions(str(kmer_rc), genomeSeq))

    for i in positions3F:


        primerSpace_start = i + gRNA_size 
        primerSpace_end = primerSpace_start + wiggleRoom + kmerSize

        for j in positions_rc:
            if primerSpace_start <= j <= primerSpace_end:
                revPositions.append(j)
                break

    kmerPositionsR.extend(revPositions)
    
    if len(kmerPositionsR) < len(kmerPositionsF):
#         print("F and R are different!")
        kmerPositionsF = kmerPositionsF[:-1]
        positions3F = positions3F[:-1]

#     print("Number of R primer Positions: ",len(kmerPositionsR))


    kmerPositionDiff = positionDifference(kmerPositionsR, positions3F)



    primerSeqF = initialPrimerSeqF(kmerPositionsF)
    primerSeqR = initialPrimerSeqR(kmerPositionsR)
    
    gcFractionR = []
    gcFractionF = []
    for currPrimer in primerSeqF:
        gcFractionF.append(gcContentFrac(currPrimer))
    for currPrimer in primerSeqR:
        gcFractionR.append(gcContentFrac(currPrimer))

    primerInitialSeq = {'primerF sequence': primerSeqF, 'primerR sequence': primerSeqR,
                        'kmer index': kmerPositionsF,
                        'primerF 3': positions3F, 'primerR 3': kmerPositionsR, 
                        'position difference': kmerPositionDiff, 'gcFracF': gcFractionF, 'gcFracR': gcFractionR}

    if len(primerInitialSeq) == 0:
        return pd.DataFrame()
    
    df_primerSeq = pd.DataFrame(primerInitialSeq)
    df_primerSeq.head()

    df_primerSeq_filtered = df_primerSeq


    #Adapter to add to the forward primers
    adaptPrimerF = 'gaaatTAATACGACTCACTATAGGG'
    # adaptGuide = 'GTTTTAGTCCCCTTCGTTTTTGGGGTAGTCTAAATCCCCTATAGTGAGTCGTATTAatttc'

#     df_primerSeq_filtered['primerF sequence'] = adaptPrimerF + df_primerSeq_filtered["primerF sequence"].astype(str)

    fastaSeqOccupied = np.zeros(len(fastaSeq), dtype=int)
#     print("Filtering sequence:", chosenKmers)
    
    #Screening primers to see if kmer binding sites exist
    for currKmer in chosenKmers:
#         
        kmer_rc = str(revComplementSeq(currKmer))
        df_primerSeq_filtered["Fexclude"]=df_primerSeq_filtered["primerF sequence"].str.find(kmer_rc)
        df_primerSeq_filtered["Rexclude"]=df_primerSeq_filtered["primerR sequence"].str.find(kmer_rc)
        df_primerSeq_filtered = df_primerSeq_filtered[df_primerSeq_filtered.Fexclude < 0]
        df_primerSeq_filtered = df_primerSeq_filtered[df_primerSeq_filtered.Rexclude < 0]
        
      
    if df_primerSeq_filtered.shape[0] == 0:
        return pd.DataFrame()
    
    for index, row in df_primerSeq_filtered.iterrows():
        currF = row['primerF 3']
        currR = row['primerR 3']

        df_primerSeq_filtered.loc[index,'ampUnique'] = sum(fastaSeqUnique[currF:currR-28])
        df_primerSeq_filtered.loc[index,'fUnique10'] = sum(fastaSeqUnique[currF-28-5:currF-28])
        df_primerSeq_filtered.loc[index,'rUnique10'] = sum(fastaSeqUnique[currR:currR+5])
        df_primerSeq_filtered.loc[index,'amplicon'] = fastaSeq[currF:currR]
        if sum(fastaSeqOccupied[currF-20:currR+20]>0):
            df_primerSeq_filtered.loc[index,'occupied'] = 2
        else:
            df_primerSeq_filtered.loc[index,'occupied'] = 1
            fastaSeqOccupied[currF-20:currR+20] = np.ones(len(range(currF-20,currR+20)))

        primerF = row['primerF sequence']
        primerR = row['primerR sequence']
    
    
    df_primerSeq_filtered = df_primerSeq_filtered[df_primerSeq_filtered.gcFracF<0.7]
    df_primerSeq_filtered = df_primerSeq_filtered[df_primerSeq_filtered.gcFracR<0.7]    
    df_primerSeq_filtered = df_primerSeq_filtered[df_primerSeq_filtered.ampUnique>0]
    df_primerSeq_filtered = df_primerSeq_filtered[df_primerSeq_filtered.fUnique10>0]
    df_primerSeq_filtered = df_primerSeq_filtered[df_primerSeq_filtered.rUnique10>0]
    df_primerSeq_filtered = df_primerSeq_filtered[df_primerSeq_filtered.occupied<2]
    
#     print("Remaining primer pairs:", df_primerSeq_filtered.shape)
        
    return df_primerSeq_filtered


In [9]:
# Run this snippet to expand an existing primer pool (made with pre-defined kmers) by one additional k-mer

numBootStrap = 10 #This value determines the number of random searches done at each pool size


numPrimers = [] 
numKmers = 1 #Number of new kmers added at each step of pool expansion

for currNum in range(numBootStrap):
    kmerBootStrap = ['AAAGA','ACCAA','AGCTC','CGATG','GCAGC','GCTCA','GGCCG','GGCTT','GGTGT','GTACT','GTGGT']
    bootSample = random.sample(range(0,len(chosenKmers)), numKmers)
    for i in bootSample:
        kmerBootStrap.append(chosenKmers[int(i)])
    try:
        df_boot = numCompatiblePrimers(kmerBootStrap, gRNA_size, wiggleRoom, primerLength, genomeSeq=fastaSeq) 
#         except:
#             df_boot = pd.DataFrame()
        if df_boot.shape[0]>0:
            print(df_boot.shape[0])
            print(kmerBootStrap)
        numPrimers.append(df_boot.shape[0])
    except:
        continue

# kmerStarterDict.loc[x,'numPrimers'] = numPrimers



In [22]:
# Run this snippet to generate a new primer pool without any pre-defined kmers

numBootStrap = 10 #This value determines the number of random searches done at each kmer pool size 
maxNumKmers = 6 #This value determines the maximum kmer pool size searched

fastaSeqUnique = np.ones(len(fastaSeq,), dtype=int)
numPrimers = []
max_shape = 0;
kmerBoot = []
best_kmer = []
print("NumKmers\tPoolSize")
for numKmers in [1]*maxNumKmers:
    
    for currNum in range(numBootStrap):
        kmerBootStrap = kmerBoot.copy()
        bootSample = random.sample(range(0,len(chosenKmers)), numKmers)
        for i in bootSample:
            kmerBootStrap.append(chosenKmers[int(i)])
        try:
            df_boot = numCompatiblePrimers(kmerBootStrap, gRNA_size, wiggleRoom, primerLength, genomeSeq=fastaSeq)          
            if df_boot.shape[0]>max_shape:
                print(len(kmerBootStrap),"\t\t", df_boot.shape[0])
                best_kmer = kmerBootStrap.copy()              
                numPrimers.append(df_boot.shape[0])
                max_shape = max(numPrimers)
        except:
            df_boot = pd.DataFrame()
    kmerBoot = best_kmer.copy()
# kmerStarterDict.loc[x,'numPrimers'] = numPrimers
print("List of kmers:", kmerBoot)


NumKmers	PoolSize
1 		 2
1 		 10
1 		 13
1 		 21
1 		 31
2 		 134
2 		 160
3 		 208
3 		 403
4 		 510
4 		 517
4 		 554
4 		 570
5 		 638
5 		 700
5 		 729
5 		 757
6 		 802
6 		 865
6 		 935
List of kmers: ['GTTGT', 'TGACG', 'TCGGT', 'CACGG', 'ATGTC', 'CGGAA']
